In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter 
import math
from scipy import stats 

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]  #Removing these stocks as these contain errors
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'A'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 1431195,
 'calculationPrice': 'close',
 'change': -1.53,
 'changePercent': -0.00999,
 'close': 0,
 'closeSource': 'licfaifo',
 'closeTime': None,
 'companyName': 'Agilent Technologies Inc.',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': '1eadmutd5e ypleiiec n r',
 'highTime': 1718576873680,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 161.11,
 'iexCloseTime': 1715193858244,
 'iexLastUpdated': 1690465553052,
 'iexMarketPercent': 0.06337043324664506,
 'iexOpen': 160.64,
 'iexOpenTime': 1725150792404,
 'iexRealtimePrice': 161.66,
 'iexRealtimeSize': 24,
 'iexVolume': 90946,
 'lastTradeTime': 1688230064015,
 'latestPrice': 158.62,
 'latestSource': 'Close',
 'latestTime': 'November 23, 2022',
 'latestUpdate': 1687626271538,
 'latestVolume': None,
 'low': 0,
 'lowSource': 'l

In [4]:
pe_ratio = data['peRatio']
pe_ratio

36.02

In [5]:

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_cols = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [7]:
final_dataframe = pd.DataFrame(columns = my_cols)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_cols), 
                                        ignore_index = True)
        
    
final_dataframe

<ipython-input-7-689031b7ee0b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-7-689031b7ee0b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-7-689031b7ee0b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-7-689031b7ee0b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-7-689031b7ee0b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,161.98,35.87,N/A
1,AAL,14.85,-6.22,N/A
2,AAP,152.22,18.09,N/A
3,AAPL,152.82,25.23,N/A
4,ABBV,165.40,21.49,N/A
...,...,...,...,...
496,YUM,127.54,29.7,N/A
497,ZBH,120.32,93.47,N/A
498,ZBRA,275.52,31.79,N/A
499,ZION,53.92,5.9,N/A


In [8]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]   # Getting rid of negative values
final_dataframe = final_dataframe[:50]                                              # Selecting the top 50 stocks
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [10]:
portfolio_input()


Enter the value of your portfolio:2500000


In [11]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,AIG,61.46,3.6,813
1,AIV,8.15,4.56,6134
2,PFG,93.44,4.61,535
3,APA,47.70,4.62,1048
4,PHM,44.80,4.76,1116
5,NUE,152.85,4.76,327
6,PVH,63.40,4.9,788
7,PSX,110.75,5.04,451
8,COF,104.41,5.04,478
9,MOS,51.00,5.17,980


In [12]:
# Now to more improve the performance of our model we use other metrics to finalize our strategy which are -

#Price-to-earnings ratio
#Price-to-book ratio
#Price-to-sales ratio
#Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
#Enterprise Value divided by Gross Profit (EV/GP)

In [13]:
symbol = 'A'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()          # Here the endpoint for the other metrics is found from iex cloud doc

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']   # Here advanced stats is the endpoint for this ratio from iex cloud

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [14]:
robust_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

robust_dataframe = pd.DataFrame(columns = robust_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        robust_dataframe = robust_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = robust_columns),
            ignore_index = True
        )

<ipython-input-14-4bfbafe62e1b>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  robust_dataframe = robust_dataframe.append(
<ipython-input-14-4bfbafe62e1b>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  robust_dataframe = robust_dataframe.append(
<ipython-input-14-4bfbafe62e1b>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  robust_dataframe = robust_dataframe.append(
<ipython-input-14-4bfbafe62e1b>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  robust_dataframe = robust_dataframe.append(
<ipython-input-14-4bfbafe62e1b>:38: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version

In [15]:
robust_dataframe[robust_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,191.000,N/A,60.49,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,310.990,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,317.100,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,95.590,N/A,48.49,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
121,CTXS,108.200,N/A,41.85,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
135,DISCK,24.580,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
164,ETFC,49.640,N/A,14.67,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
185,FLIR,58.890,N/A,32.51,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
189,FOX,30.320,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [17]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    robust_dataframe[column].fillna(robust_dataframe[column].mean(), inplace = True)

In [18]:
robust_dataframe[robust_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [19]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in robust_dataframe.index:
    for metric in metrics.keys():
        robust_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(robust_dataframe[metric], robust_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(robust_dataframe[metric])

#Print the entire DataFrame    
robust_dataframe

0      0.804391
1      0.041916
2      0.407186
3      0.585828
4      0.491018
         ...   
496    0.674651
497    0.964072
498    0.708583
499    0.091816
500    0.780439
Name: PE Percentile, Length: 501, dtype: object
0      0.856287
1      0.065868
2      0.499002
3       0.98004
4      0.938124
         ...   
496    0.061876
497    0.321357
498    0.672655
499    0.243513
500    0.926148
Name: PB Percentile, Length: 501, dtype: object
0      0.866267
1      0.011976
2      0.123752
3      0.843313
4      0.780439
         ...   
496    0.801397
497    0.620758
498    0.453094
499    0.368263
500    0.906188
Name: PS Percentile, Length: 501, dtype: object
0        0.9002
1      0.806387
2      0.289421
3      0.760479
4      0.411178
         ...   
496    0.802395
497    0.746507
498    0.602794
499    0.071856
500    0.876248
Name: EV/EBITDA Percentile, Length: 501, dtype: object
0       0.88024
1      0.021956
2       0.11976
3      0.914172
4      0.688623
         ...   
4

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,155.87,N/A,36.24,0.804391,9.44,0.856287,7.2000,0.866267,25.632079,0.9002,13.064476,0.88024,N/A
1,AAL,14.99,N/A,-6.15,0.041916,-1.21,0.065868,0.2163,0.011976,20.752697,0.806387,0.757252,0.021956,N/A
2,AAP,153.80,N/A,18.31,0.407186,3.25,0.499002,0.8246,0.123752,9.997242,0.289421,2.067146,0.11976,N/A
3,AAPL,152.11,N/A,25.10,0.585828,47.93,0.98004,6.3100,0.843313,18.935993,0.760479,14.873425,0.914172,N/A
4,ABBV,160.97,N/A,21.68,0.491018,18.45,0.938124,5.0700,0.780439,12.700444,0.411178,8.727949,0.688623,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,130.89,N/A,28.63,0.674651,-4.21,0.061876,5.3400,0.801397,20.671757,0.802395,9.578208,0.734531,N/A
497,ZBH,119.82,N/A,90.94,0.964072,2.08,0.321357,3.6100,0.620758,18.590406,0.746507,6.210671,0.457086,N/A
498,ZBRA,281.39,N/A,30.59,0.708583,5.35,0.672655,2.5500,0.453094,15.332318,0.602794,6.133209,0.449102,N/A
499,ZION,51.75,N/A,5.94,0.091816,1.85,0.243513,2.1000,0.368263,4.648104,0.071856,2.131418,0.121756,N/A


In [ ]:
#We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter stocks.

#The RV Score is the arithmetic mean of the 4 percentile scores that we calculated .

In [21]:
from statistics import mean

for row in robust_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(robust_dataframe.loc[row, metrics[metric]])
    robust_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
robust_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,155.87,N/A,36.24,0.804391,9.44,0.856287,7.2000,0.866267,25.632079,0.9002,13.064476,0.88024,0.861477
1,AAL,14.99,N/A,-6.15,0.041916,-1.21,0.065868,0.2163,0.011976,20.752697,0.806387,0.757252,0.021956,0.189621
2,AAP,153.80,N/A,18.31,0.407186,3.25,0.499002,0.8246,0.123752,9.997242,0.289421,2.067146,0.11976,0.287824
3,AAPL,152.11,N/A,25.10,0.585828,47.93,0.98004,6.3100,0.843313,18.935993,0.760479,14.873425,0.914172,0.816766
4,ABBV,160.97,N/A,21.68,0.491018,18.45,0.938124,5.0700,0.780439,12.700444,0.411178,8.727949,0.688623,0.661876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,130.89,N/A,28.63,0.674651,-4.21,0.061876,5.3400,0.801397,20.671757,0.802395,9.578208,0.734531,0.61497
497,ZBH,119.82,N/A,90.94,0.964072,2.08,0.321357,3.6100,0.620758,18.590406,0.746507,6.210671,0.457086,0.621956
498,ZBRA,281.39,N/A,30.59,0.708583,5.35,0.672655,2.5500,0.453094,15.332318,0.602794,6.133209,0.449102,0.577246
499,ZION,51.75,N/A,5.94,0.091816,1.85,0.243513,2.1000,0.368263,4.648104,0.071856,2.131418,0.121756,0.179441


In [22]:
robust_dataframe.sort_values(by = 'RV Score', inplace = True)
robust_dataframe = robust_dataframe[:50]
robust_dataframe.reset_index(drop = True, inplace = True)

In [23]:
portfolio_input()

Enter the value of your portfolio:2500000


In [24]:
position_size = float(portfolio_size) / len(robust_dataframe.index)
for i in range(0, len(robust_dataframe['Ticker'])-1):
    robust_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / robust_dataframe['Price'][i])
robust_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,33.10,1510,6.46,0.10978,0.8489,0.085828,0.2045,0.007984,4.263256,0.063872,1.131115,0.057884,0.06507
1,PVH,62.50,800,4.68,0.067864,0.7920,0.081836,0.4486,0.055888,4.880447,0.081836,1.040762,0.043912,0.066267
2,AIG,62.05,805,3.70,0.05988,1.2200,0.130739,0.7901,0.117764,2.170449,0.01996,0.778265,0.023952,0.070459
3,GM,40.22,1243,7.08,0.125749,0.9160,0.093812,0.4013,0.047904,2.105194,0.017964,1.622704,0.093812,0.075848
4,UNM,41.38,1208,7.29,0.133733,0.9460,0.095808,0.6863,0.095808,3.843195,0.041916,0.658118,0.015968,0.076647
5,BA,178.17,280,-12.63,0.035928,-6.1000,0.05988,1.7700,0.303393,-27.633916,0.005988,-174.267995,0.001996,0.081437
6,COF,102.72,486,5.16,0.077844,0.7913,0.077844,1.0900,0.170659,3.716411,0.033932,1.262667,0.067864,0.085629
7,C,48.30,1035,6.96,0.116766,0.5390,0.071856,1.0500,0.159681,4.044330,0.053892,1.101117,0.055888,0.091617
8,F,14.32,3491,6.38,0.107784,1.3900,0.150699,0.3829,0.043912,4.233685,0.061876,1.879694,0.105788,0.094012
9,SYF,37.72,1325,6.01,0.093812,1.4100,0.157685,0.9562,0.143713,3.505098,0.02994,1.096771,0.053892,0.095808


In [ ]:
# We will use the xlsxWriter library to create Excel files.

In [26]:
writer = pd.ExcelWriter('quant_value_strategy.xlsx', engine='xlsxwriter')
robust_dataframe.to_excel(writer, sheet_name='Quant Value Strategy', index = False)

In [27]:
# Now we'll create formats for the columns in excel file
# String format for tickers
# $XX.XX format for stock prices
# $XX,XXX format for market capitalization
# Integer format for the number of shares to purchase
# Float formats with 1 decimal for each valuation metric

In [28]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [30]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Quant Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Quant Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [31]:
writer.save()